In [ ]:
# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit

# the code below configures memory growth mode, so that we can more easily see how much VRAM our model needs
# one doesn't need to run this cell to run the optimization

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt
from ECD_control.ECD_optimization.tf_adam_optimizer import AdamOptimizer
from ECD_control.gate_sets.GRAPE import GRAPE
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt

In [ ]:
q_dim = 4
c_dim = 20
DAC_time_resolution = 2
fock = 4

# In GHz = cycles / ns
anharm = .4
kerr = 1e-5
chi = 2e-3
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi) * ad * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(b + bd), 1j*D*(b - bd), D*(a + ad), 1j*D*(a - ad)]

init_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0))
]

final_states = [
    qt.tensor(qt.basis(c_dim, fock), qt.basis(q_dim, 0))
]

In [ ]:
# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'pulse_delta_t' : DAC_time_resolution,
    'inplace' : True, # true uses less memory, but is slower
    'scale' : 1.0 # range of DAC amplitudes for initial random waves
}
GRAPE_gate_set = GRAPE(**gate_set_params)

In [ ]:
#the optimization options
synth_params = {
    'gateset' : GRAPE_gate_set,
    'N_blocks': 300, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 3, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'GRAPE Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.best_fidelity()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize(logdir='logs')

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs